<a href="https://colab.research.google.com/github/fazeci/QA_with_rag/blob/main/QA_with_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import datetime
import numpy as np

# Normal dağılım kullanarak yanıt boyutunu üretme
def generate_normal_distribution_size(mean, std_dev):
    return max(0, int(np.random.normal(mean, std_dev)))

# Rastgele bir tarih oluşturma (son 30 gün içinde)
def generate_random_timestamp():
    end_time = datetime.datetime.now()
    start_time = end_time - datetime.timedelta(days=30)
    random_time = start_time + (end_time - start_time) * random.random()
    return random_time

# Apache log formatına uygun sahte veriler oluşturma
def generate_fake_apache_log_entry():
    ip = ".".join(map(str, random.sample(range(1, 256), 4)))
    timestamp = generate_random_timestamp()
    method = random.choice(['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'OPTIONS', 'PATCH', 'CONNECT', 'TRACE'])
    uri = random.choice(['/index.html', '/about.html', '/contact.html', '/login', '/products', '/api/data', '/admin'])
    protocol = "HTTP/1.1"
    status = random.choice([200, 201, 204, 301, 302, 304, 400, 401, 403, 404, 500, 502, 503, 504])
    size = generate_normal_distribution_size(mean=1500, std_dev=500)

    log_entry = f'{ip} - - [{timestamp.strftime("%d/%b/%Y:%H:%M:%S +0000")}] "{method} {uri} {protocol}" {status} {size}'
    return log_entry

# Belirli bir sayıda sahte log verisi oluşturma ve dosyaya yazma
def generate_and_save_fake_logs(num_entries, filename='fake_apache_logs.txt'):
    fake_logs = [generate_fake_apache_log_entry() for _ in range(num_entries)]

    with open(filename, 'w') as file:
        for entry in fake_logs:
            file.write(entry + '\n')

# Kullanım
generate_and_save_fake_logs(1000)

# İlk 1000 log verisini görüntüleme
with open('fake_apache_logs.txt', 'r') as file:
    logs_to_display = [next(file) for _ in range(10)]
    print("".join(logs_to_display))


38.49.82.243 - - [06/Aug/2024:02:02:23 +0000] "OPTIONS /products HTTP/1.1" 302 490
191.135.70.33 - - [09/Aug/2024:22:31:04 +0000] "GET /login HTTP/1.1" 502 1895
87.78.32.79 - - [03/Aug/2024:11:31:52 +0000] "GET /contact.html HTTP/1.1" 500 1927
169.219.90.71 - - [18/Jul/2024:23:58:13 +0000] "CONNECT /index.html HTTP/1.1" 400 1912
73.6.108.103 - - [18/Jul/2024:01:27:52 +0000] "GET /admin HTTP/1.1" 200 1240
216.37.84.199 - - [05/Aug/2024:21:23:39 +0000] "POST /login HTTP/1.1" 302 1854
211.239.226.126 - - [17/Jul/2024:18:35:50 +0000] "OPTIONS /index.html HTTP/1.1" 401 2260
15.175.236.243 - - [05/Aug/2024:16:35:36 +0000] "PUT /login HTTP/1.1" 301 1577
159.160.84.146 - - [09/Aug/2024:23:23:38 +0000] "PUT /products HTTP/1.1" 200 947
119.225.64.219 - - [17/Jul/2024:10:15:12 +0000] "GET /contact.html HTTP/1.1" 404 1021



In [2]:
import pandas as pd
import re

# Log dosyasını oku
log_file_path = '/content/fake_apache_logs.txt'

# Apache/Nginx log formatını regex ile ayıklama
log_pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+)\s+\S+\s+\S+\s+\[(?P<timestamp>.*?)\]\s+"(?P<method>\S+)\s+(?P<uri>\S+)\s+(?P<protocol>\S+)"\s+(?P<status>\d+)\s+(?P<size>\d+)'

# Logları DataFrame'e yükle
log_df = pd.read_csv(log_file_path, sep='|', names=['log_entry'])
log_df = log_df['log_entry'].str.extract(log_pattern)

# Temel analiz
print(log_df.head())
print(log_df.info())


              ip                   timestamp   method            uri  \
0   38.49.82.243  06/Aug/2024:02:02:23 +0000  OPTIONS      /products   
1  191.135.70.33  09/Aug/2024:22:31:04 +0000      GET         /login   
2    87.78.32.79  03/Aug/2024:11:31:52 +0000      GET  /contact.html   
3  169.219.90.71  18/Jul/2024:23:58:13 +0000  CONNECT    /index.html   
4   73.6.108.103  18/Jul/2024:01:27:52 +0000      GET         /admin   

   protocol status  size  
0  HTTP/1.1    302   490  
1  HTTP/1.1    502  1895  
2  HTTP/1.1    500  1927  
3  HTTP/1.1    400  1912  
4  HTTP/1.1    200  1240  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ip         1000 non-null   object
 1   timestamp  1000 non-null   object
 2   method     1000 non-null   object
 3   uri        1000 non-null   object
 4   protocol   1000 non-null   object
 5   status     1000 non-nu

In [3]:
# Veri türlerini dönüştürme
log_df['timestamp'] = pd.to_datetime(log_df['timestamp'], format='%d/%b/%Y:%H:%M:%S +0000')
log_df['status'] = log_df['status'].astype(int)
log_df['size'] = log_df['size'].astype(int)

print(log_df.dtypes)


ip                   object
timestamp    datetime64[ns]
method               object
uri                  object
protocol             object
status                int64
size                  int64
dtype: object


In [4]:
# Geçersiz (olumsuz) status kodları veya size değerlerini kontrol etme
invalid_status = log_df[~log_df['status'].between(100, 599)]  # HTTP status kodları 100-599 aralığındadır
invalid_size = log_df[log_df['size'] < 0]  # Boyutun negatif olmaması gerekir

print(f"Geçersiz status kayıtları: {len(invalid_status)}")
print(f"Geçersiz size kayıtları: {len(invalid_size)}")


Geçersiz status kayıtları: 0
Geçersiz size kayıtları: 0


In [6]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [38]:
!pip install faiss-cpu
import faiss

In [48]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Modeli yükle
model = SentenceTransformer('all-MiniLM-L6-v2')

qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")


# Log verisini birleştirerek metin formatında kullanıma hazırlama
log_df['log_text'] = log_df.apply(lambda row: f"{row['method']} {row['uri']} {row['status']}", axis=1)

# Vektörlere dönüştürme
log_vectors = model.encode(log_df['log_text'].tolist(), show_progress_bar=True)

# FAISS ile vektörleri bir veri tabanında depolama
index = faiss.IndexFlatL2(log_vectors.shape[1])  # L2 mesafe metriği
# Vektörleri endekse ekleme
index.add(np.array(log_vectors, dtype='float32'))


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [62]:

def get_related_logs(query):
    """
    Kullanıcı sorgusuna en benzer log kayıtlarını bulur.
    """
    query_vec = model.encode([query], show_progress_bar=False)
    distances, indices = index.search(np.array(query_vec, dtype='float32'), k=4)
    return log_df.iloc[indices[0]]


def generate_answer(query, related_logs):
    """
    `roberta-base` veya başka bir model kullanarak benzer log kayıtlarından bir yanıt oluşturur.
    """
    if related_logs.empty:
        return "No similar log entries found."

    # Bağlamı oluşturma
    context = "\n".join([
        f"On {row['timestamp']}, a {row['method']} request to {row['uri']} resulted in a {row['status']} status code."
        for _, row in related_logs.iterrows()
    ])

    print("Context for the question:\n", context)  # Bağlamı kontrol et

    # Yanıt oluşturma
    result = qa_pipeline(
        question=query,
        context=context,
        max_length=150,  # Uzunluğu ihtiyaca göre ayarlayın
        num_beams=5,
        early_stopping=True
    )

    return result.get('answer', 'Unable to generate an answer.')

def main():
    while True:
        query = input("Enter your query ('q' to quit): ")
        if query.lower() == 'q':
            break

        related_logs = get_related_logs(query)
        answer = generate_answer(query, related_logs)

        print("\nQuery:", query)
        if not related_logs.empty:
            print("Related Logs:\n", related_logs.to_string(index=False))
        else:
            print("No similar logs found.")
        print("Answer:", answer)

if __name__ == "__main__":
    main()


Enter your query ('q' to quit): when was the last time code 200 was received?
Context for the question:
 On 2024-08-10 17:09:34, a PATCH request to /products resulted in a 200 status code.
On 2024-08-09 17:10:56, a PATCH request to /products resulted in a 200 status code.
On 2024-08-14 11:46:47, a GET request to /products resulted in a 201 status code.
On 2024-08-09 21:16:30, a PATCH request to /products resulted in a 201 status code.

Query: when was the last time code 200 was received?
Related Logs:
             ip           timestamp method       uri protocol  status  size            log_text
  97.8.232.102 2024-08-10 17:09:34  PATCH /products HTTP/1.1     200  1269 PATCH /products 200
107.34.149.152 2024-08-09 17:10:56  PATCH /products HTTP/1.1     200  2360 PATCH /products 200
 73.58.182.238 2024-08-14 11:46:47    GET /products HTTP/1.1     201  2134   GET /products 201
104.198.158.45 2024-08-09 21:16:30  PATCH /products HTTP/1.1     201  1630 PATCH /products 201
Answer: 2024-08-0

In [ ]:
#when was the last time code 200 was received?
#What was the most common URI in the last month?
#What was the most common HTTP method used in the last week?
#What is the status of the latest log entry?
#What was the most common error code in the last week?
#Which method has the highest average request size?
#What is the average response time for each HTTP status code?
#what people do most on this site
#Which HTTP method was used most frequently in the last week?
#What was the most common error code in the logs for the past month?
#When was the last time a 500 error code occurred?
#Which URI received the most requests in the last 24 hours?


In [47]:
# En sık kullanılanlarıanaliz etme
counts = log_df['status'].value_counts()
print(counts)


status
302    85
201    82
404    77
304    77
504    77
500    73
503    73
403    72
502    71
204    65
400    64
200    64
301    63
401    57
Name: count, dtype: int64


**Proje Raporu: Log Verileri Üzerinde RAG Modeli ile Sorgu Yanıtlama Sistemi**

***1. Giriş***

*Projenin Amacı:*
Bu proje, web trafik log verilerini kullanarak kullanıcı sorgularına uygun yanıtlar veren bir Retrieval-Augmented Generation (RAG) sistemi geliştirmeyi amaçlamaktadır. Sistem, log verileri üzerinde arama yaparak en uygun kayıtları bulmakta ve bu kayıtları kullanarak jeneratif bir dil modeli ile kullanıcıya yanıt oluşturmaktadır.

*Kullanılan Veri Seti:*
- Apache log verileri
- 100,000 satırlık örnek log kayıtları (IP adresleri, erişilen sayfalar, zaman damgaları vb.)

***2. Metodoloji***

*Aşama 1: Veri Hazırlığı ve Ön İşleme*
- Veri Analizi: Log verileri incelendi ve gerekli sütunlar (IP adresi, URI, HTTP metodu, durum kodu vb.) ayıklandı.
- Veri Temizleme: Eksik veya hatalı veriler temizlendi ve sütunlar uygun veri türlerine dönüştürüldü.
- Vektörleştirme: Log kayıtları, dil modeli kullanılarak vektörlere dönüştürüldü. FAISS kütüphanesi ile bu vektörler sorgulanabilir bir veri tabanına yüklendi.

*Aşama 2: RAG Modelinin Kurulumu*
- Bilgi Alma Modülü: FAISS kütüphanesi kullanılarak, kullanıcıdan gelen sorguya en benzer log kayıtlarını bulmak üzere bir bilgi alma modülü oluşturuldu.
- Jeneratif Modül:`T5` modelini kullanarak, bulunan log kayıtları üzerinde anlamlı yanıtlar oluşturmak için bir jeneratif modül geliştirildi.
- Sistem Entegrasyonu: Bilgi alma ve jeneratif modülleri entegre edilerek, tam işlevsel bir RAG modeli oluşturuldu.

*Aşama 3: Sistem Entegrasyonu ve Test*
- Sistemin Test Edilmesi: Çeşitli kullanıcı sorguları ile sistem test edildi. Yanıtların doğruluğu ve kalitesi değerlendirildi.
- Sistem Performansı: Modelin performansı, sorgu süresi ve yanıt kalitesi açısından ölçüldü.

***3. Karşılaşılan Zorluklar ve Çözümleri***

- Zorluk: Log verilerinin karmaşıklığı ve tekrarlayan bilgiler.
  
  Çözüm:Veriyi vektörleştirme sürecinde, benzer log kayıtlarını daha iyi işleyebilmek için veri temizliği ve özetleme teknikleri uygulandı.
  
- Zorluk: Modelin tekrarlayan veya anlamsız yanıtlar üretmesi.
  
  Çözüm: Modelde `no_repeat_ngram_size` ve `max_new_tokens` parametreleri ayarlandı. Ayrıca, giriş kontekstinin daha anlamlı hale getirilmesi sağlandı.

- Zorluk: Veri Türleri ile İlgili Sorunlar
  
  Çözüm: Veri türlerini uygun hale getirmek için öncelikle her bir sütunun içeriği analiz edildi ve uygun veri türlerine dönüştürüldü.
  
Zaman Damgaları: timestamp sütunu, datetime formatına dönüştürüldü, böylece zaman bazlı gruplama ve analizler yapılabildi.

Durum Kodları ve Yanıt Boyutları: status ve size sütunları, int64 veri türüne dönüştürüldü, böylece sayısal işlemler ve karşılaştırmalar yapılabildi.

IP Adresleri: IP adresleri, string formatında bırakıldı, ancak gerektiğinde IP’lerin sınıflandırılması ve gruplandırılması için düzenli ifadeler (regex) kullanıldı.

***4. Performans Değerlendirmesi***

-Doğruluk: Sistem, kullanıcı sorgularına yanıt verirken doğru ve alakalı log kayıtlarını kullandı. Yanıtlar, log kayıtlarının genel paternlerine uygun şekilde oluşturuldu.
-Yanıt Süresi: Sistem, büyük veri setiyle çalışmasına rağmen makul sürelerde yanıt üretebildi. FAISS vektör veri tabanı, yüksek verimli arama sağladı.


***5. Sonuç ve Öneriler***

* Genel Değerlendirme: Geliştirilen sistem, log verileri üzerinde başarılı bir
şekilde sorgu yapabilen ve yanıtlar üretebilen bir RAG modeli olarak çalıştı.

* Öneriler: Daha farklı veri türleriyle sistemin test edilmesi, sistemin genelleme yeteneğini artırabilir.

* BART, roberta-base ve T5 dil modelleri denendi ancak yanıtların doğruluğunun artırılması için modelin daha çok geliştirilmesi gerekmektedir.

***6. Kaynaklar***

- Kullanılan kütüphaneler: `transformers`, `sentence-transformers`, `faiss`, `pandas`, `numpy`



FATMA ZEHRA ÇINAR
